In [1]:
import pandas as pd
import numpy as np

# Importing dataset


In [2]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset

spotify-million-song-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# %unzip spotify-million-song-dataset.zip

In [4]:
spotifyDF = pd.read_csv("spotify_millsongdata.csv")

# Milestone 3

In [5]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

c:\Users\malak\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
spotifyDFMS3=spotifyDF.copy()

In [7]:
def split_sentences(df, col_name):
    df['X'] = df[col_name].apply(lambda x: ' '.join(x.split()[:len(x.split())//2]))
    df['Y'] = df[col_name].apply(lambda x: ' '.join(x.split()[len(x.split())//2:]))
    return df

spotifyDFMS3=split_sentences(spotifyDFMS3,"text")
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","Look at her face, it's a wonderful face And it...",And when we go for a walk in the park And she ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","Take it easy with me, please Touch me gently l...",your eyes Like the feeling of a thousand butte...


In [8]:
print(len(spotifyDFMS3["Y"][0])+len(spotifyDFMS3["X"][0]))
print(len(spotifyDFMS3["text"][0]))

719
781


There is a difference in the values since the split_sentences method also removes the extra unneeded tokens such as white spaces adn \n.

In [9]:
spotifyDFMS3['artist_lyric']=spotifyDFMS3["artist"]+" || "+spotifyDFMS3["X"]

In [10]:
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y,artist_lyric
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","Look at her face, it's a wonderful face And it...",And when we go for a walk in the park And she ...,"ABBA || Look at her face, it's a wonderful fac..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","Take it easy with me, please Touch me gently l...",your eyes Like the feeling of a thousand butte...,"ABBA || Take it easy with me, please Touch me ..."


In [11]:
def max_words(df, col_name):
    max_word_count = df[col_name].apply(lambda x: len(x.split(" "))).max()
    return max_word_count

max_word_count_X = max_words(spotifyDFMS3, 'X')
max_word_count_Y = max_words(spotifyDFMS3, 'Y')
max_word_count = max_words(spotifyDFMS3, 'artist_lyric')
print("X max :", max_word_count_X)
print("Y max :", max_word_count_Y)
print("X AND artist max:",max_word_count)

X max : 413
Y max : 414
X AND artist max: 416


In [12]:
from datasets import load_dataset
spotifyDataset = load_dataset('csv',data_files="spotify_millsongdata.csv", split="train")

In [13]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [14]:
def split_sentences(df,OldDF):
    x=[]
    y=[]
    artist_lyric=[]
    for string in OldDF["X"]:
        x.append(string)
    for string in OldDF["Y"]:
        y.append(string)
    for string in OldDF["artist_lyric"]:
        artist_lyric.append(string)
    df = df.add_column("X", x)
    df = df.add_column("Y", y)
    df = df.add_column("artist_lyric", artist_lyric)
    return df

spotifyDataset=split_sentences(spotifyDataset,spotifyDFMS3)

In [15]:
spotifyDataset[0]

{'artist': 'ABBA',
 'song': "Ahe's My Kind Of Girl",
 'link': '/a/abba/ahes+my+kind+of+girl_20598417.html',
 'text': "Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n",
 'X': "Look at her face, it's a wonderful face And it mean

In [16]:
columns_to_drop = ["song", "link","text"]
spotifyDataset = spotifyDataset.remove_columns(columns_to_drop)

In [17]:
spotifyDataset[0]

{'artist': 'ABBA',
 'X': "Look at her face, it's a wonderful face And it means something special to me Look at the way that she smiles when she sees me How lucky can one fellow be? She's just my kind of girl, she makes me feel fine Who could ever believe that she could be mine? She's just my kind of girl, without her I'm blue And if she ever leaves me what could I do, what could I do?",
 'Y': "And when we go for a walk in the park And she holds me and squeezes my hand We'll go on walking for hours and talking About all the things that we plan She's just my kind of girl, she makes me feel fine Who could ever believe that she could be mine? She's just my kind of girl, without her I'm blue And if she ever leaves me what could I do, what could I do?",
 'artist_lyric': "ABBA || Look at her face, it's a wonderful face And it means something special to me Look at the way that she smiles when she sees me How lucky can one fellow be? She's just my kind of girl, she makes me feel fine Who could 

In [18]:
spotifyDataset

Dataset({
    features: ['artist', 'X', 'Y', 'artist_lyric'],
    num_rows: 57650
})

In [19]:
def process_dataset(example):
    artist = example['artist']
    first_half = example['X']
    second_half = example['Y']
    
    input_text = f"{artist}: {first_half}"
    target_text = second_half
    
    return {'input_text': input_text, 'target_text': target_text}

spotifyDataset = spotifyDataset.map(process_dataset, remove_columns=["X", "Y", "artist_lyric"])

In [20]:
spotifyDataset

Dataset({
    features: ['artist', 'input_text', 'target_text'],
    num_rows: 57650
})

In [21]:
def tokenize_function(examples):
    inputs = [f"<s> {text} </s>" for text in examples['input_text']]
    targets = [f"<s> {text} </s>" for text in examples['target_text']]
    model_inputs = tokenizer(inputs, max_length=max_word_count, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_word_count, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    decoder_input_ids = labels['input_ids'].copy()
    decoder_start_token_id = tokenizer.pad_token_id
    
    for idx in range(len(decoder_input_ids)):
        decoder_input_ids[idx] = [decoder_start_token_id] + decoder_input_ids[idx][:-1]
    
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs

spotifyDataset = spotifyDataset.map(tokenize_function, batched=True, remove_columns=["artist", "input_text", "target_text"])

Map: 100%|██████████| 57650/57650 [00:24<00:00, 2372.02 examples/s]


In [22]:
spotifyDataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 57650
})

In [23]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [24]:
spotifyDataset=spotifyDataset.train_test_split(test_size=0.2)

In [25]:
spotifyDataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 46120
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 11530
    })
})

In [26]:
train_dataset = spotifyDataset["train"].with_format("numpy")[:] 
test_dataset = spotifyDataset["test"].with_format("numpy")[:]

In [27]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [29]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [32]:
model.fit(train_dataset, validation_data=test_dataset, epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

 8/24 [=========>....................] - ETA: 58:20 - loss: 10.3900 - sparse_categorical_accuracy: 0.1490  

In [28]:
# import tensorflow as tf
# from tensorflow import keras

# optimizer = keras.optimizers.Adam(learning_rate=5e-5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [35]:
# def convert_to_tf_dataset(hf_dataset):
#     return hf_dataset.to_tf_dataset(
#         columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
#         shuffle=True,
#     )

# train_dataset = convert_to_tf_dataset(spotifyDataset['train'])
# eval_dataset = convert_to_tf_dataset(spotifyDataset['test'])

In [40]:
# def train_step(batch):
#     with tf.GradientTape() as tape:
#         outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], decoder_input_ids=batch['decoder_input_ids'], labels=batch['labels'])
#         loss = outputs.loss
#         logits = outputs.logits
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     accuracy_metric.update_state(batch['labels'], logits)
    
#     return loss

In [41]:
# def eval_step(batch):
#     outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], decoder_input_ids=batch['decoder_input_ids'], labels=batch['labels'])
#     loss = outputs.loss
#     logits = outputs.logits
#     accuracy_metric.update_state(batch['labels'], logits)
#     return loss

In [42]:
# num_epochs = 3
# train_steps = len(train_dataset)
# eval_steps = len(eval_dataset)

In [ ]:
# for epoch in range(num_epochs):
#     # Training
#     print('TRAINING')
#     print(f"Epoch {epoch+1}/{num_epochs}")
#     total_loss = 0.0
#     accuracy_metric.reset_states()
#     for step, batch in enumerate(train_dataset):
#         loss = train_step(batch)
#         total_loss += tf.reduce_mean(loss).numpy()
#         if (step + 1) % 100 == 0:
#             print(f"Step {step+1}/{train_steps}, Loss: {total_loss / (step+1):.4f}, Accuracy: {accuracy_metric.result().numpy():.4f}")
    
#     # Evaluation
#     print('EVALUATION')
#     total_loss = 0.0
#     accuracy_metric.reset_states()
#     for step, batch in enumerate(eval_dataset):
#         loss = eval_step(batch)
#         total_loss += tf.reduce_mean(loss).numpy()
    
#     print(f"Validation Loss: {total_loss / eval_steps:.4f}, Validation Accuracy: {accuracy_metric.result().numpy():.4f}")